In [1]:
# pip install PyMuPDF Pillow
# pip install pytesseract and pip install tesseract
# ref https://www.geeksforgeeks.org/reading-text-from-the-image-using-tesseract/
# ref https://www.kaggle.com/code/dhorvay/pytesseract-multiple-languages
# ref https://stackoverflow.com/questions/50951955/pytesseract-tesseractnotfound-error-tesseract-is-not-installed-or-its-not-i
# code adjusted as per https://stackoverflow.com/questions/75321414/python-page-object-has-no-attribute-getimagelist-where-i-try-to-extract-im

 
import io 

import cv2
import fitz
import os,argparse
from PIL import Image 
from tqdm.notebook import tqdm

# install pytesseract in windows : https://github.com/UB-Mannheim/tesseract/wiki
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 
# tessdata_dir_config = '--tessdata-dir "C:\Program Files\Tesseract-OCR\tessdata"'

C:\Users\avitr\anaconda3\envs\test_env\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [23]:
path = r"D:\OneDrive - NITT\Custom_Download\71Lpqdrpi4L.jpg"

In [14]:
    
file = "நெஞ்சை_உருக்கும்_நீதிக்கதைகள்.pdf"

pdf_file = fitz.open(file) 

for page_index in tqdm(range(len(pdf_file))): 

    page = pdf_file[page_index] 
    image_list = page.get_images() 

    for image_index, img in enumerate(page.get_images(), start=1): 
        xref = img[0] 
        base_image = pdf_file.extract_image(xref) 
        image_bytes = base_image["image"] 
        image = Image.open(io.BytesIO(image_bytes)) 
        
        text = pytesseract.image_to_string(image) 
        if 0 <= len(text) < 210: #'\x0c'
            image_ext = base_image["ext"] 
            image_name = f"page{page_index + 1}_image{image_index}.{image_ext}"
            image.save('IMG/'+image_name) 
            print(image_name,len(text))

  0%|          | 0/106 [00:00<?, ?it/s]

page1_image1.jpeg 0
page2_image1.jpeg 0
page2_image2.jpeg 0
page3_image1.jpeg 61
page5_image1.jpeg 77


KeyboardInterrupt: 

In [24]:
image = Image.open(path)

In [25]:
# image = Image.open(io.BytesIO(image_bytes)) 
        
text = pytesseract.image_to_string(image) 
# if 0 <= len(text) < 210: #'\x0c'
#     image_ext = base_image["ext"] 
#     image_name = f"page{page_index + 1}_image{image_index}.{image_ext}"
#     image.save('IMG/'+image_name) 
#     print(image_name,len(text))

In [26]:
text

'All day snack\nfor boosting energy\n\nAll Natural\n\nIngredients\nJuantum &\nImproves “Neturals Quick source\nDigestion f of Energy\nee eee Rich in fibre\n\nDelicious in\n\ntaste and vitamins\n\naT FOR\nsTiO\n\nContains superfood in\ncombination with Millets\nand Multigrains\n'

In [12]:
text

'17 cm/ 7 inch\n\nYdul ¢°¢ /WD FL\n'

In [ ]:
import re
import constants
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

def common_mistake(unit):
    if unit in constants.allowed_units:
        return unit
    if unit.replace('ter', 'tre') in constants.allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in constants.allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    s_stripped = "" if s==None or str(s)=='nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in constants.allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, constants.allowed_units))
    return number, unit


def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)
    
    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)
        